# Photometric detrending

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def gaussian(theta, x): 
    """
    Gaussian signal
    """
    amp, x0, sigma = theta
    return amp * np.exp(-0.5 * (x0 - x)**2 / sigma**2)

In [ ]:
np.random.seed(42)

# Set the number of data points
n = 1000

times = np.arange(n)

# Create position vectors for the centroid of the target
x = 10 * np.random.randn(n) + 500
y = 12 * np.random.randn(n) + 500

# Create sinusoidal varying background with time
background_period = 150
background = (100 * np.random.randn(n) + 1000 + 
              300 * np.sin(2 * np.pi / background_period * times) + 
              0.2 * times)

# Create astrophysical signal, which has a Gaussian shape
signal = gaussian([-500, 400, 50], times) + 50 * np.random.randn(n)

err = 50 * np.ones(n)

# Plot the vectors of the centroid positions, the time-varying 
# background, and the signal 
fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].plot(x, y, 'k.')
ax[0].set(xlabel='X centroid', ylabel='Y centroid')
ax[1].plot(background, 'k.')
ax[1].set(xlabel='Time', ylabel='Background flux')
ax[2].plot(times, signal, 'k.')
ax[2].set(xlabel='Time', ylabel='Astrophysical Signal')
fig.tight_layout()

In [ ]:
np.random.seed(1984)

# Create a "hidden" relationship between each of the 
# input vectors (position, background, plus signal) 
# and the output vector (observed flux)

X_true = np.vstack([
    times - times.mean(),
    x - x.mean(), 
    y - y.mean(), 
    (x - x.mean()) * (y - y.mean()), 
    background, 
    signal
]).T

# Assign random weights to each of the vectors
betas_soln = np.random.random(size=X_true.shape[1])

flux_observed = X_true @ betas_soln

plt.plot(times, flux_observed, 'k.')
plt.gca().set(xlabel='Times', ylabel='Observed Flux')

In [ ]:
def linreg(X, flux, err):
    """
    Solve linear regression such that `X betas = flux`
    """
    inv_N = np.linalg.inv(np.diag(err)**2)
    betas = np.linalg.inv(X.T @ inv_N @ X) @ X.T @ inv_N @ flux
    cov = np.linalg.inv(X.T @ inv_N @ X)
    return betas, cov

In [ ]:
X_obs = np.vstack([
    # Trend in time
    times - times.mean(),
    
    # Positional terms
    x - x.mean(), 
    y - y.mean(),
    (x - x.mean()) * (y - y.mean()),
    
    # Background terms
    np.sin(2 * np.pi / background_period * times), 
    np.cos(2 * np.pi / background_period * times),
    np.ones(n),
    
    # Signal model
    gaussian([-500, 400, 50], times)
]).T

betas, cov = linreg(X_obs, flux_observed, err)

best_flux_model = X_obs @ betas

fig, ax = plt.subplots(2, 1, figsize=(6, 8), sharex=True)
ax[0].plot(times, flux_observed, 'k.', label='Observed')
ax[0].plot(times, best_flux_model, 'r', label='Model')
ax[0].legend(loc='lower right')
ax[0].set(ylabel='Flux')

ax[1].plot(times, flux_observed - best_flux_model, 'k.')
ax[1].set(xlabel='Time', ylabel='Residuals')
plt.show()

In [ ]:
plt.plot(times, signal, 'k.', label='Input signal')
plt.plot(times, flux_observed - (X_obs[:, :-1] @ betas[:-1]), 'r.', label='Recovered signal')
plt.legend(loc='lower right')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.show()